In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('wine_quality').getOrCreate()

In [0]:
data_white = spark.read.csv('/FileStore/tables/winequality_white.csv',inferSchema='True',header='true',sep=';')
data_red = spark.read.csv('/FileStore/tables/winequality_red.csv',inferSchema='True',header='true',sep=';')

In [0]:
data_white.head(1)

Out[4]: [Row(fixed acidity=7.0, volatile acidity=0.27, citric acid=0.36, residual sugar=20.7, chlorides=0.045, free sulfur dioxide=45.0, total sulfur dioxide=170.0, density=1.001, pH=3.0, sulphates=0.45, alcohol=8.8, quality=6)]

In [0]:
data_red.head(1)

Out[5]: [Row(fixed acidity=7.4, volatile acidity=0.7, citric acid=0.0, residual sugar=1.9, chlorides=0.076, free sulfur dioxide=11.0, total sulfur dioxide=34.0, density=0.9978, pH=3.51, sulphates=0.56, alcohol=9.4, quality=5)]

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
data_white.columns

Out[7]: ['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality']

In [0]:
data_red.columns

Out[8]: ['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality']

In [0]:
assembler = VectorAssembler(inputCols=['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol'], outputCol='features')

In [0]:
output_white = assembler.transform(data_white)

In [0]:
output_red = assembler.transform(data_red)

In [0]:
output_white.printSchema()

root
-- fixed acidity: double (nullable = true)
-- volatile acidity: double (nullable = true)
-- citric acid: double (nullable = true)
-- residual sugar: double (nullable = true)
-- chlorides: double (nullable = true)
-- free sulfur dioxide: double (nullable = true)
-- total sulfur dioxide: double (nullable = true)
-- density: double (nullable = true)
-- pH: double (nullable = true)
-- sulphates: double (nullable = true)
-- alcohol: double (nullable = true)
-- quality: integer (nullable = true)
-- features: vector (nullable = true)

In [0]:
from pyspark.ml.classification import DecisionTreeClassifier, GBTClassifier

In [0]:
dtc_white = DecisionTreeClassifier(labelCol='quality',featuresCol='features')

In [0]:
dtc_red = DecisionTreeClassifier(labelCol='quality',featuresCol='features')

In [0]:
final_data_white = output_white.select('features','quality')
final_data_red = output_red.select('features','quality')

In [0]:
train_data_white,test_data_white = final_data_white.randomSplit([0.7,0.3])

In [0]:
train_data_red,test_data_red = final_data_red.randomSplit([0.7,0.3])

In [0]:
dtc_model_white = dtc_red.fit(train_data_white)

In [0]:
dtc_model_red = dtc_red.fit(train_data_red)

In [0]:
dtc_preds_white = dtc_model_red.transform(test_data_white)

In [0]:
dtc_preds_red = dtc_model_red.transform(test_data_red)

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [0]:
my_binary_eval_white = BinaryClassificationEvaluator(labelCol='quality')
my_binary_eval_red = BinaryClassificationEvaluator(labelCol='quality')

In [0]:
print('Decision Tree Clasifier Results - White Wine:')
print(my_binary_eval_white.evaluate(dtc_preds_white))

Decision Tree Clasifier Results - White Wine:
1.0

In [0]:
print('Decision Tree Clasifier Results - Red Wine:')
print(my_binary_eval_red.evaluate(dtc_preds_red))

Decision Tree Clasifier Results - Red Wine:
1.0